# topic model for first period

In [1]:
import pandas as pd
c1 = pd.read_csv("D:\\CODE\\data set for QTA\\c1_tokens.csv")

In [2]:
# go through each row and split the tokens by comma
c1t = [t.split(", ") for t in c1['tokens']] 

remove Chinese stop-words

In [ ]:
import jieba
from jieba.analyse import set_stop_words

# load stop words table
def load_stopwords(path='cn_stopwords.txt'):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return {line.strip() for line in f}
    except FileNotFoundError:
        return {'error'} 

In [4]:
from tqdm import tqdm

def filter_stopwords(tokenized_docs, stopwords):
    """带进度条的过滤函数"""
    return [
        [token for token in doc if token not in stopwords and len(token) > 1]
        for doc in tqdm(tokenized_docs, desc="过滤进度", unit="doc")
    ]

if __name__ == "__main__":
    stopwords = load_stopwords()
    
    # 执行过滤并显示进度
    cleaned_c1t = filter_stopwords(c1t, stopwords)
    
    # 结果对比
    print("原始数据示例:", c1t[0])
    print("清洗后示例:", cleaned_c1t[0])

过滤进度: 100%|██████████| 268232/268232 [00:18<00:00, 14457.36doc/s]

原始数据示例: ['国民党', '当局', '破坏', '荷泽', '协议', '蓄意', '放', '水淹', '我', '解放区', '不顾', '七百万', '人民', '生命', '图逞', '内战', '阴谋', '中共中央', '发言人', '表示', '坚决', '反对', '新华社', '延安', '十日', '电', '记者', '顷以', '国民党', '当局', '破坏', '荷泽', '治河', '协议', '阴谋', '淹毙', '黄河', '故道', '豫', '冀鲁', '三省', '七百万', '同胞', '事', '访问', '中共中央', '发言人', '发言人', '谈话', '如', '次', '问', '黄河', '问题', '的', '经过', '如何', '答', '黄河', '原经', '豫北', '冀南', '斜贯', '山东', '至', '利津入', '海', '民国', '二十七年', '夏', '始由', '国民党', '当局', '派', '军队', '在', '河南', '郑州', '开封', '间', '中牟县', '境', '之', '花园口', '实行', '决堤', '图', '以此', '阻止', '敌人', '的', '进攻', '自此', '黄河', '乃', '由', '花园口', '东南', '流至豫皖', '边', '入', '淮河', '转', '运河', '长江', '入海', '黄河', '决口', '的', '结果', '有百害而无一利', '加以', '国民党', '当局', '事前', '对', '人民', '保守', '秘密', '致', '黄河', '所经', '之', '地', '无数', '同胞', '之生', '命', '财产', '惨遭', '损失', '为', '抗战史', '中', '极', '惨痛', '之', '一页', '而', '淮河', '运河', '的', '水道', '亦', '因而', '遭受', '影响', '河床', '有', '逐年', '淤浅', '之虞', '现', '抗战', '结束', '以', '充分', '人力物力', '与', '友邦', '援助', '疏导', '黄河', '使', '复归', '故道', '原则上', 

tranform data into Document - term Matrix (DTM)

In [ ]:
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary

# 基于全量数据重建词典（避免词汇丢失）
full_dictionary = Dictionary(cleaned_c1t)
full_dictionary.filter_extremes(no_below=10, no_above=0.5)

# 生成全量文档-词矩阵
dtm = [full_dictionary.doc2bow(doc) for doc in cleaned_c1t]


train LDA model with 15 topics

In [ ]:
from gensim.models import LdaMulticore
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

k = 15 

lda_model = LdaMulticore(
    corpus=dtm,
    id2word=full_dictionary,
    num_topics=k,
    passes=15,
    alpha=1,
    eta=0.1,
    workers=4
)

# 打印模型训练完成提示
print("LDA 模型训练完成！")

# 保存模型到文件
model_path = "D:\\CODE\\data set for QTA\\lda_model 1946-1976.model"
lda_model.save(model_path)
print(f"LDA 模型已保存到文件：{model_path}")

Show the 10 main words of each topics

In [ ]:

# 显示每个主题的主要单词
num_words = 10  # 每个主题显示的单词数量
topics = lda_model.show_topics(num_topics=k, num_words=num_words, formatted=False)

# 打印每个主题及其主要单词
for topic_id, topic_words in topics:
    print(f"主题 {topic_id}:")
    print(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
    print()

# 将主要单词保存到文件
output_file = "lda_topics 1.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for topic_id, topic_words in topics:
        f.write(f"主题 {topic_id}:\n")
        f.write(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
        f.write("\n\n")

print(f"主题主要单词已保存到文件：{output_file}")


主题 0:
我们 (0.0265), 他们 (0.0137), 一个 (0.0132), 问题 (0.0122), 就是 (0.0113), 什么 (0.0108), 没有 (0.0102), 这个 (0.0098), 阶级斗争 (0.0087), 这样 (0.0085)

主题 1:
教育 (0.0279), 理论 (0.0191), 农村 (0.0191), 知识青年 (0.0186), 学校 (0.0169), 学习 (0.0164), 马克思主义 (0.0155), 学生 (0.0132), 他们 (0.0121), 工农兵 (0.0117)

主题 2:
地区 (0.0243), 全省 (0.0116), 工程 (0.0109), 今年 (0.0103), 建设 (0.0094), 各地 (0.0088), 农田 (0.0084), 施工 (0.0083), 地方 (0.0070), 万亩 (0.0068)

主题 3:
大队 (0.0430), 大寨 (0.0329), 公社 (0.0321), 社员 (0.0300), 农业 (0.0269), 贫下中农 (0.0221), 生产 (0.0208), 党支部 (0.0182), 发展 (0.0151), 粮食 (0.0130)

主题 4:
会议 (0.0320), 代表 (0.0205), 政府 (0.0194), 李先念 (0.0194), 中华人民共和国 (0.0163), 阁下 (0.0123), 国家 (0.0113), 会谈 (0.0110), 问题 (0.0107), 外长 (0.0096)

主题 5:
复辟 (0.0367), 改悔 (0.0222), 他们 (0.0177), 卫生 (0.0150), 一九七 (0.0140), 医院 (0.0137), 人民 (0.0121), 外事 (0.0114), 治疗 (0.0106), 病人 (0.0084)

主题 6:
人民 (0.0734), 修正主义 (0.0461), 我们 (0.0255), 中国 (0.0230), 斗争 (0.0135), 民族 (0.0124), 国家 (0.0120), 发展 (0.0106), 团结 (0.0095), 帝国主义 (0.0090)

主题 7:
日本 (0.0800), 妇女 (0.0

# topic model for rest of 2 periods

### c2

In [1]:
import pandas as pd
c2 = pd.read_csv("D:\\CODE\\data set for QTA\\c2_tokens.csv")

# go through each row and split the tokens by comma
c2t = [t.split(", ") for t in c2['tokens']] 


In [2]:

import jieba
from jieba.analyse import set_stop_words

# 加载停用词表（推荐哈工大停用词表）
def load_stopwords(path='cn_stopwords.txt'):
    """加载停用词表并返回集合"""
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return {line.strip() for line in f}
    except FileNotFoundError:
        # 基础停用词保底
        return {'wsgsb'} 
    

In [3]:
from tqdm import tqdm

def filter_stopwords(tokenized_docs, stopwords):
    """带进度条的过滤函数"""
    return [
        [token for token in doc if token not in stopwords and len(token) > 1]
        for doc in tqdm(tokenized_docs, desc="过滤进度", unit="doc")
    ]

if __name__ == "__main__":
    stopwords = load_stopwords()
    
    # 执行过滤并显示进度
    cleaned_c2t = filter_stopwords(c2t, stopwords)  # 修改点：c1t→c2t，cleaned_c1t→cleaned_c2t[1,2](@ref)
    
    # 结果对比
    print("原始数据示例:", c2t[0])  # 修改点：c1t→c2t
    print("清洗后示例:", cleaned_c2t[0])

过滤进度: 100%|██████████| 217603/217603 [00:07<00:00, 28110.40doc/s]

原始数据示例: ['乘胜前进', '人民日报', '红旗', '杂志', '解放军报', '一九七七年', '元旦', '社论', '全国', '人民', '在', '深入', '揭批', '四人帮', '的', '战斗', '中', '满怀', '胜利', '的', '豪情', '喜气洋洋', '地', '迎来', '了', '新', '的', '一年', '一九七六年', '是', '极', '不', '平凡', '的', '一年', '是', '我们', '取得', '了', '伟大', '的', '历史性', '胜利', '的', '一年', '这', '一年', '我们', '失去', '了', '伟大', '的', '领袖', '和', '导师', '毛泽东', '主席', '失去', '了', '敬爱', '的', '周恩来', '总理', '和', '朱德', '委员长', '我们', '遭受', '了', '地震', '等', '严重', '的', '自然灾害', '在', '我们', '党', '面临', '巨大', '困难', '的', '情况', '下', '华国锋', '同志', '为首', '的', '党中央', '领导', '我们', '一举', '粉碎', '了', '王张', '江姚', '反党集团', '篡党夺权', '的', '阴谋', '经过', '这场', '惊心动魄', '的', '阶级', '大', '搏斗', '中国', '革命', '避免', '了', '一次', '大', '曲折', '中国', '历史', '避免', '了', '一次', '大', '倒退', '中国', '人民', '避免', '了', '一场', '大灾难', '我们', '的', '胜利', '是', '毛泽东思想', '的', '伟大', '胜', '利', '是', '毛主席', '的', '无产阶级', '革命', '路线', '的', '伟大胜利', '全党全军', '全国', '各族人民', '为', '我们', '又', '有', '了', '自己', '的', '英明领袖', '华国锋', '主席', '感到', '幸福', '和', '自豪', '除掉', '四害', '人民', '大', '解放', '思想', '大', 

In [6]:
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary

# 基于全量数据重建词典（避免词汇丢失）
dictionary = Dictionary(cleaned_c2t)
dictionary.filter_extremes(no_below=10, no_above=0.5)

# 生成全量文档-词矩阵
dtm = [dictionary.doc2bow(doc) for doc in cleaned_c2t]


In [ ]:
from gensim.models import LdaMulticore
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 配置 LDA 模型参数（保持不变）
num_topics = 15
passes = 15
k = 15  

lda_model = LdaMulticore(
    corpus=dtm,
    id2word=dictionary,
    num_topics=k,
    passes=15,
    alpha=1,
    eta=0.1,
    workers=4
)

# 打印模型训练完成提示
print("LDA 模型训练完成！")

# 保存模型到文件
model_path = "D:\\CODE\\data set for QTA\\lda_model 1977-1989.model"
lda_model.save(model_path)
print(f"LDA 模型已保存到文件：{model_path}")

LDA 模型训练完成！
LDA 模型已保存到文件：D:\CODE\data set for QTA\lda_model 1977-1989.model


In [10]:

# 显示每个主题的主要单词
num_words = 10  # 每个主题显示的单词数量
topics = lda_model.show_topics(num_topics=k, num_words=num_words, formatted=False)

# 打印每个主题及其主要单词
for topic_id, topic_words in topics:
    print(f"主题 {topic_id}:")
    print(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
    print()

# 将主要单词保存到文件
output_file = "lda_topics 2.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for topic_id, topic_words in topics:
        f.write(f"主题 {topic_id}:\n")
        f.write(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
        f.write("\n\n")

print(f"主题主要单词已保存到文件：{output_file}")

主题 0:
研究 (0.0173), 我国 (0.0171), 会议 (0.0169), 工作 (0.0105), 专家 (0.0089), 科技 (0.0089), 技术 (0.0072), 代表 (0.0067), 进行 (0.0066), 全国 (0.0063)

主题 1:
工程 (0.0132), 建设 (0.0098), 建成 (0.0058), 铁路 (0.0055), 万吨 (0.0055), 我国 (0.0054), 安全 (0.0050), 汽车 (0.0047), 设计 (0.0047), 运输 (0.0047)

主题 2:
记者 (0.0209), 台湾 (0.0162), 中国 (0.0156), 香港 (0.0152), 北京 (0.0149), 报道 (0.0144), 新闻 (0.0130), 上海 (0.0120), 新华社 (0.0105), 公司 (0.0097)

主题 3:
企业 (0.0284), 职工 (0.0146), 公司 (0.0127), 实行 (0.0098), 管理 (0.0089), 万元 (0.0086), 改革 (0.0079), 单位 (0.0078), 厂长 (0.0076), 领导 (0.0067)

主题 4:
教育 (0.0203), 学生 (0.0154), 工作 (0.0150), 青年 (0.0124), 学校 (0.0121), 他们 (0.0112), 学习 (0.0103), 文化 (0.0103), 活动 (0.0098), 教师 (0.0084)

主题 5:
一个 (0.0121), 他们 (0.0110), 我们 (0.0105), 自己 (0.0082), 没有 (0.0078), 人们 (0.0052), 这样 (0.0050), 一位 (0.0048), 这个 (0.0045), 现在 (0.0039)

主题 6:
同志 (0.0195), 北京 (0.0098), 中国 (0.0097), 委员 (0.0077), 主席 (0.0073), 参加 (0.0073), 文化 (0.0070), 举行 (0.0069), 代表 (0.0067), 全国 (0.0066)

主题 7:
体育 (0.0264), 比赛 (0.0202), 中国 (0.0143), 选手

### c3

In [1]:
import pandas as pd
c3 = pd.read_csv("D:\\CODE\\data set for QTA\\c3_tokens.csv")

In [2]:
# 删除包含 nan 值的行
c3 = c3.dropna(subset=['tokens'])
print(c3['tokens'].dtype)

c3t = [t.split(", ") for t in c3['tokens']] 
print(c3t[0])

object
['社论', '满怀信心', '迎接', '九十年代', '伴随', '着', '年', '第一记', '钟声', '二十世纪', '九十年代', '来到', '了', '回顾过去', '十年', '的', '征程', '展望未来', '十年', '的', '前景', '我们', '满怀豪情', '充满信心', '刚刚', '过去', '的', '八十年代', '是', '我国', '各族人民', '在', '中国共产党', '的', '领导', '下', '沿着', '社会主义', '道路', '胜利', '前进', '的', '年代', '是', '社会主义', '建设', '事业', '和', '社会主义', '改革', '事业', '取得', '辉煌成就', '的', '年代', '是', '社会主义', '在', '中国', '大', '地上', '生机勃勃', '显示', '了', '强大', '生命力', '的', '年代', '在', '八十年代', '我们', '以', '经济', '建设', '为', '中心', '坚持', '四项', '基本', '原则', '坚持', '改革开放', '社会主义', '现代化', '建设', '取得', '了', '举世瞩目', '的', '成就', '国民经济', '迅速', '增长', '国民', '生产总值', '翻了一番', '已经', '上升', '到', '居', '世界', '第', '位', '教育', '科学', '文化', '事业', '和', '国防建设', '都', '取得', '了', '巨大', '成绩', '综合国力', '显著', '增强', '人民', '生活', '明显改善', '在', '八十年代', '我们', '坚持', '四项', '基本', '原则', '党的领导', '更加', '坚强', '社会主义', '制度', '更加', '巩固', '经过', '一次', '又', '一次', '的', '斗争', '特别', '是', '去年', '春', '夏之交', '制止动乱', '和', '平息', '反革命', '暴乱', '的', '斗争', '击退', '了', '资产阶级', '自由化', '的', '进攻', '粉碎', '了', '国

In [3]:

import jieba
from jieba.analyse import set_stop_words

# 加载停用词表（推荐哈工大停用词表）
def load_stopwords(path='cn_stopwords.txt'):
    """加载停用词表并返回集合"""
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return {line.strip() for line in f}
    except FileNotFoundError:
        # 基础停用词保底
        return {'wsgsb'} 

In [4]:
from tqdm import tqdm

def filter_stopwords(tokenized_docs, stopwords):
    """带进度条的过滤函数"""
    return [
        [token for token in doc if token not in stopwords and len(token) > 1]
        for doc in tqdm(tokenized_docs, desc="过滤进度", unit="doc")
    ]

if __name__ == "__main__":
    stopwords = load_stopwords()
    
    # 执行过滤并显示进度
    cleaned_c3t = filter_stopwords(c3t, stopwords)  # 修改点：c1t→c2t，cleaned_c1t→cleaned_c2t[1,2](@ref)
    
    # 结果对比
    print("原始数据示例:", c3t[0])  # 修改点：c1t→c2t
    print("清洗后示例:", cleaned_c3t[0])

过滤进度: 100%|██████████| 217912/217912 [00:07<00:00, 28468.87doc/s]

原始数据示例: ['社论', '满怀信心', '迎接', '九十年代', '伴随', '着', '年', '第一记', '钟声', '二十世纪', '九十年代', '来到', '了', '回顾过去', '十年', '的', '征程', '展望未来', '十年', '的', '前景', '我们', '满怀豪情', '充满信心', '刚刚', '过去', '的', '八十年代', '是', '我国', '各族人民', '在', '中国共产党', '的', '领导', '下', '沿着', '社会主义', '道路', '胜利', '前进', '的', '年代', '是', '社会主义', '建设', '事业', '和', '社会主义', '改革', '事业', '取得', '辉煌成就', '的', '年代', '是', '社会主义', '在', '中国', '大', '地上', '生机勃勃', '显示', '了', '强大', '生命力', '的', '年代', '在', '八十年代', '我们', '以', '经济', '建设', '为', '中心', '坚持', '四项', '基本', '原则', '坚持', '改革开放', '社会主义', '现代化', '建设', '取得', '了', '举世瞩目', '的', '成就', '国民经济', '迅速', '增长', '国民', '生产总值', '翻了一番', '已经', '上升', '到', '居', '世界', '第', '位', '教育', '科学', '文化', '事业', '和', '国防建设', '都', '取得', '了', '巨大', '成绩', '综合国力', '显著', '增强', '人民', '生活', '明显改善', '在', '八十年代', '我们', '坚持', '四项', '基本', '原则', '党的领导', '更加', '坚强', '社会主义', '制度', '更加', '巩固', '经过', '一次', '又', '一次', '的', '斗争', '特别', '是', '去年', '春', '夏之交', '制止动乱', '和', '平息', '反革命', '暴乱', '的', '斗争', '击退', '了', '资产阶级', '自由化', '的', '进攻', '粉碎', '了', '

In [5]:
from sklearn.model_selection import train_test_split
from gensim.corpora import Dictionary

# 基于全量数据重建词典（避免词汇丢失）
dictionary = Dictionary(cleaned_c3t)
dictionary.filter_extremes(no_below=10, no_above=0.5)

# 生成全量文档-词矩阵
dtm = [dictionary.doc2bow(doc) for doc in cleaned_c3t]

In [6]:
from gensim.models import LdaMulticore
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 配置 LDA 模型参数（保持不变）
num_topics = 15
passes = 15
k = 15  

lda_model = LdaMulticore(
    corpus=dtm,
    id2word=dictionary,
    num_topics=k,
    passes=15,
    alpha=1,
    eta=0.1,
    workers=4
)

# 打印模型训练完成提示
print("LDA 模型训练完成！")

# 保存模型到文件
model_path = "D:\\CODE\\data set for QTA\\lda_model 1977-1989.model"
lda_model.save(model_path)
print(f"LDA 模型已保存到文件：{model_path}")

LDA 模型训练完成！
LDA 模型已保存到文件：D:\CODE\data set for QTA\lda_model 1977-1989.model


In [7]:

# 显示每个主题的主要单词
num_words = 10  # 每个主题显示的单词数量
topics = lda_model.show_topics(num_topics=k, num_words=num_words, formatted=False)

# 打印每个主题及其主要单词
for topic_id, topic_words in topics:
    print(f"主题 {topic_id}:")
    print(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
    print()

# 将主要单词保存到文件
output_file = "lda_topics 3.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for topic_id, topic_words in topics:
        f.write(f"主题 {topic_id}:\n")
        f.write(", ".join([f"{word} ({weight:.4f})" for word, weight in topic_words]))
        f.write("\n\n")

print(f"主题主要单词已保存到文件：{output_file}")

主题 0:
工程 (0.0189), 建设 (0.0152), 城市 (0.0083), 西部 (0.0070), 记者 (0.0061), 要闻 (0.0057), 开发 (0.0056), 铁路 (0.0054), 公路 (0.0050), 环境 (0.0048)

主题 1:
工作 (0.0252), 建设 (0.0209), 发展 (0.0179), 重要 (0.0112), 思想 (0.0093), 我们 (0.0093), 社会主义 (0.0086), 加强 (0.0081), 问题 (0.0075), 领导 (0.0074)

主题 2:
群众 (0.0133), 干部 (0.0128), 就业 (0.0100), 职工 (0.0097), 服务 (0.0091), 他们 (0.0082), 万元 (0.0081), 领导 (0.0074), 工作 (0.0072), 全市 (0.0064)

主题 3:
公司 (0.0210), 北京 (0.0138), 国际 (0.0123), 中国 (0.0115), 上海 (0.0112), 旅游 (0.0101), 本报讯 (0.0097), 技术 (0.0095), 我国 (0.0089), 经济 (0.0084)

主题 4:
教育 (0.0193), 部队 (0.0125), 科技 (0.0110), 政治 (0.0084), 学生 (0.0083), 研究 (0.0078), 学校 (0.0076), 社会 (0.0076), 学习 (0.0073), 军队 (0.0070)

主题 5:
我们 (0.0115), 一个 (0.0107), 他们 (0.0095), 自己 (0.0074), 没有 (0.0061), 就是 (0.0047), 人们 (0.0040), 这里 (0.0037), 这样 (0.0037), 一位 (0.0036)

主题 6:
部门 (0.0109), 规定 (0.0101), 有关 (0.0092), 工作 (0.0086), 管理 (0.0082), 法律 (0.0069), 进行 (0.0062), 单位 (0.0056), 监督 (0.0055), 检查 (0.0051)

主题 7:
美国 (0.0156), 中国 (0.0127), 问题 (0.0106), 